In [1]:
import cx_Oracle

In [1]:
username = 'admin'
password = 'password'
dsn = 'cis550pricetracker.cgcukgcycu5f.us-east-1.rds.amazonaws.com/CIS550DB'
port = 1512
encode = 'UTF-8'

In [6]:
connection = None
try:
    connection = cx_Oracle.connect(
        username,
        password,
        dsn)

    # show the version of the Oracle Database
    print('yay')
    print(connection.version)
    connection.cursor
         
except cx_Oracle.Error as error:
    print(error)
    connection.close()

c = connection.cursor() 
c.execute('WITH Sears AS (SELECT (SHLDQ/AMOUNT) as adjusted_sears FROM (SELECT dateID, SHLDQ FROM Stocks s WHERE dateID = 20060323) a JOIN Dates d ON a.dateID = d.dateID \
JOIN Inflation i ON d.Year = i.Year), \
Valentino AS (SELECT (Valentino/AMOUNT) as adjusted_valentino \
FROM (SELECT dateID, Valentino \
FROM Commodities c \
WHERE dateID = 20060323) a \
JOIN Dates d ON a.dateID = d.dateID \
JOIN Inflation i ON d.Year = i.Year) \
SELECT GREATEST(v.adjusted_valentino, s.adjusted_sears) as higher_value \
FROM Sears s, Valentino v;') 
for i in c:
    print(i)

SyntaxError: EOL while scanning string literal (<ipython-input-6-9d34a7a2b983>, line 21)

In [139]:
class Cart:
    # Represents total cash in the bank
    total_cash = 100000.00
    # Dictionary of all stocks/items in portfolio
    # (Share, Date) -> (Number of Shares, Share Price)
    portfolio = {}

    # num_shares - int, share_price (infl. adj) - float, transaction - string "sell/buy"
    # Return True - Transaction successful, total update
    # Return False - Transaction failed, not enough money
    def updateTotal(self, num_shares, share_price, transaction):
        trans_amount = num_shares * share_price
        if transaction == "sell":
            new_total_cash = self.total_cash + trans_amount
            self.total_cash = new_total_cash
            return True
        else:
            print('buy')
            new_total_cash = self.total_cash - trans_amount
            # This means transaction price is too high
            if (new_total_cash < 0):
                return False
            else:
                print('update')
                self.total_cash = new_total_cash
                return True
    
    # share_name - string, share_date - int, num_shares - int, share_price - float
    # Return True - Transaction successful, portfolio update
    # Return False - Transaction unsuccessful, portfolio won't update
    # Assume that all inputs are correct
    def addPortfolio(self, share_name, share_date, num_shares, share_price):
        # First make the tuple for the portfolio dictionary
        key_tuple = (share_name, share_date)
        # Make sure the total is valid - must return true (only buy in add portfolio)
        # This also helps us update the bank
        success = self.updateTotal(num_shares, share_price, "buy")
        if success:
            # Check if share/date is already in dict, then update value
            if key_tuple in self.portfolio.keys():
                curr_num_shares = self.portfolio[key_tuple][0]
                new_num_shares = curr_num_shares + num_shares
                # Replace tuple - share price is technically the same!
                value_tuple = (new_num_shares, share_price)
                self.portfolio[key_tuple] = value_tuple
                return True
            # Else make a new key->value pairing
            else:
                value_tuple = (num_shares, share_price)
                self.portfolio[key_tuple] = value_tuple
                return True

    # share_name - string, share_date - int, num_shares - int, share_price - float
    # Return True - Transaction successful, portfolio update
    # Return False - Transaction unsuccessful, portfolio won't update
    # Assume that all inputs are correct
    def removePortfolio(self, share_name, share_date, num_shares, share_price):
        # First make the tuple for the portfolio dictionary
        key_tuple = (share_name, share_date)
        if key_tuple in self.portfolio.keys():
            # Update the cash total
            success = self.updateTotal(num_shares, share_price, "sell")
            # Check how many 
            curr_num_shares = self.portfolio[key_tuple][0]
            print(curr_num_shares)
            new_num_shares = curr_num_shares - num_shares
            # Delete key if no shares left
            if new_num_shares == 0:
                del self.portfolio[key_tuple] 
                return True
            # Update it otherwise 
            else:
                value_tuple = (new_num_shares, share_price)
                self.portfolio[key_tuple] = value_tuple
                return True
        # Else not possible to remove anything!
        else:
            return False


    # Add functions to get total worth and curent worth of portfolio

    def returnTotalCash(self):
        return self.total_cash
    
    def returnPortfolio(self):
        return self.portfolio


In [140]:
cart = Cart()

In [141]:
cart.addPortfolio("AAPL", 20200409, 5, 270.80)

buy
update


True

In [142]:
print(cart.portfolio)

{('AAPL', 20200409): (5, 270.8)}


In [143]:
print(cart.total_cash)

98646.0


In [144]:
cart.addPortfolio("AAPL", 20200407, 5, 270.80)

False

In [145]:
print(cart.portfolio)

{('AAPL', 20200409): (5, 270.8)}


In [146]:
print(cart.total_cash)

98646.0


In [147]:
cart.removePortfolio("AAPL", 20200407, 1, 270.80)

False

In [148]:
print(cart.total_cash)

98646.0
